In [8]:
from __future__ import print_function

# import pyLDAVis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

# import MySQLdb to use the connection later
import MySQLdb

# import re for regex
import re

# import sklearn vectorizer and LDA library
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# import pandas for dataframe
import pandas as pd

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords');

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# supress warnings on sklearn LDA
import warnings
warnings.filterwarnings("ignore")

In [10]:
connec = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     passwd="user1991",  # your password
                     db="tia_test")        # name of the data base

posts_df = pd.read_sql('select * from posts;', con=connec)
posts_doc = posts_df['post_content']

In [11]:
# custom stopword list based on understanding of the dataset
#stop_word_list = ["salary", "cum", "up", "to", "mnc", "mncs", "job", "id", "etc", "bonus", "per", "month", "aws", "days",
#                 "immediate", "position", "positions", "temp", "temporary", "perm", "permanant", "urgent", "sea", 
#                  "singapore", "apac", "asia", "pacific", "ot", "weekend", "weekday"]
stop_word_list = ["the", "be", "to", "of", "and", "a", "in", "that", "have", "i", "it", "for", "not", "on", "with", "he", "as", 
                  "you", "do", "at", "this", "but", "his", "by", "from", "they", "we", "say", "her", "she", "or", "an", "will", 
                  "my", "one", "all", "would", "there", "their", "what", "so", "up", "out", "if", "about", "who", "get", 
                  "which", "go", "me", "when", "make", "can", "like", "time", "no", "just", "him", "know", "take", "people", 
                  "into", "year", "your", "good", "some", "could", "them", "see", "other", "than", "then", "now", "look", 
                  "only", "come", "its", "over", "think", "also", "back", "after", "use", "two", "how", "our", "work", "first", 
                  "well", "way", "even", "new", "want", "because", "any", "these", "give", "day", "most", "us", 'https', 'www', 
                  'crystal', 'etc', 'is', 'youre', 'were', 'im', 'hes', 'shes', 'theyre', 'are', 'yes', 'hi', 'isnt', 'wasnt', 'hey', 
                  'co']

stops = list(stopwords.words('english'))

def treat_text(text):
    # decode text to work with special characters
    text = text.decode('utf8')
    
    # lower case the words
    text = text.lower()
    
    # substitution for consistency for words
    text = re.sub("e-commerce", "ecommerce", text)
    text = re.sub("go-jek", "gojek", text)
    
    # custom regexes based on text   
    # remove all symbols except letters numbers and blank spaces
    text = re.sub(r'[^A-Za-z0-9\s]+', ' ', text)
    
    text = re.sub(r'\b[0-9]+\b\s*', '', text)
    # remove stopwords based on nltk stop list and some custom words from looking at the dataset
    text = ' '.join([word for word in text.split() if word not in stop_word_list and word not in stops])
    
    # substitution for consistency for words
    text = re.sub("tia", "tech-in-asia", text)
    text = re.sub("tech in asia", "tech-in-asia", text)
    text = re.sub("data sets", "datasets", text)
    text = re.sub(" ai ", " artificial-intelligence ", text)
    text = re.sub(" ml ", " machine-learning ", text)
    text = re.sub("data science", "data-science", text)
    text = re.sub("data sciences", "data-science", text)
    text = re.sub("se asia", "se-asia", text)
    text = re.sub("startups", "startup", text)
    text = re.sub("southeast asia", "se-asia", text)
    text = re.sub("south-east asia", "se-asia", text)
    text = re.sub("facebooks", "facebook", text)
    text = re.sub("icos", "ico", text)
    
    # compress all more than 1 blank spaces into a single blank space and finally strip the trailing and leading blank spaces
    text = re.sub("\s\s+" , " ", text).strip()
    
    return text

In [12]:
# remove duplicates and apply text cleaning on entire dataframe
posts_parsed = (posts_doc).apply(treat_text)

In [13]:
tf_vectorizer = CountVectorizer(analyzer='word',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(posts_parsed)
print(dtm_tf.shape)

(900, 4189)


In [14]:
lda_tf = LatentDirichletAllocation(n_topics=20, 
                                   random_state=0,
                                   max_iter=100)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=100,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=20,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [15]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
17     34.067044        1       1 -0.001840 -0.137492
7      22.183153        1       2 -0.041665 -0.146608
14     18.101178        1       3 -0.112587 -0.102610
2      17.484196        1       4 -0.147348 -0.132407
15      2.223552        1       5 -0.242788  0.236359
1       1.893399        1       6 -0.203421 -0.028910
3       1.404748        1       7 -0.354630 -0.046992
10      0.666281        1       8  0.013268  0.159454
11      0.551667        1       9  0.041018  0.272665
13      0.361120        1      10  0.072166  0.135854
18      0.327130        1      11  0.067757  0.072593
19      0.295626        1      12  0.076225  0.079238
4       0.165395        1      13  0.092842 -0.026871
5       0.159790        1      14  0.126139  0.002618
8       0.036106        1      15  0.116864 -0.053773
0       0.017843        1      16  0.100318 -0.056716
6       0.016506        1      17  0.098952 -0.056852
12      0.015089        1      18  0.099577 -0.056516
16      0.015089        1      19  0.099577 -0.056516
9       0.015089        1      20  0.099577 -0.056516, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
3976  Default  1177.000000       users  1177.000000  30.0000  30.0000
3430  Default  1418.000000   singapore  1418.000000  29.0000  29.0000
2695  Default   950.000000     percent   950.000000  28.0000  28.0000
421   Default   688.000000  blockchain   688.000000  27.0000  27.0000
130   Default   417.000000     alibaba   417.000000  26.0000  26.0000
1145  Default   748.000000   ecommerce   748.000000  25.0000  25.0000
3975  Default   415.000000        user   415.000000  24.0000  24.0000
3355  Default   539.000000     service   539.000000  23.0000  23.0000
1987  Default   784.000000   investors   784.000000  22.0000  22.0000
3691  Default   415.000000      system   415.000000  21.0000  21.0000
1829  Default   473.000000         ico   473.000000  20.0000  20.0000
3356  Default   612.000000    services   612.000000  19.0000  19.0000
1653  Default   297.000000        grab   297.000000  18.0000  18.0000
1027  Default   525.000000   different   525.000000  17.0000  17.0000
616   Default  1028.000000       china  1028.000000  16.0000  16.0000
3917  Default   300.000000        uber   300.000000  15.0000  15.0000
3249  Default   818.000000        says   818.000000  14.0000  14.0000
2221  Default   537.000000         lot   537.000000  13.0000  13.0000
4021  Default   275.000000       video   275.000000  12.0000  12.0000
1893  Default   571.000000       india   571.000000  11.0000  11.0000
1034  Default   566.000000     digital   566.000000  10.0000  10.0000
2420  Default   655.000000       money   655.000000   9.0000   9.0000
1375  Default   278.000000    facebook   278.000000   8.0000   8.0000
3585  Default   269.000000       store   269.000000   7.0000   7.0000
3174  Default   294.000000        ride   294.000000   6.0000   6.0000
1339  Default   428.000000  experience   428.000000   5.0000   5.0000
2174  Default   244.000000        line   244.000000   4.0000   4.0000
501   Default   393.000000  businesses   393.000000   3.0000   3.0000
710   Default  1188.000000   companies  1188.000000   2.0000   2.0000
3285  Default   599.000000          se   599.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
421   Topic20     0.010594  blockchain   688.564478  -2.2832  -8.3402
3976  Topic20     0.010594       users  1177.984684  -2.8201  -8.3402
4084  Topic20     0.010594      wechat   321.576809  -1.5218  -8.3402
618   Topic20     0.010594     chinese   595.852763  -2.1386  -8.3402
1987  Topic20     0.010594   investors   784.241064  -2.4133  -8.3402
3469  Topic20     0.010594      social   458.027151  -1.8755  -8.3402
1145  Topic20     0.010594   ecommerce 